## LightGBM test (Multi-Classification)
### player情報を付加

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

In [11]:
ALL_MERGE = 'intermediate/all_merge_1.f'

In [12]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 93)

### 不要な列を削除

In [13]:
all_pitch.drop(
    columns=[
        '年度', 
        '試合ID', 
        'ホームチームID', 'アウェイチームID', 
        '投手ID', '投手チームID', 
        '打者ID', '打者チームID', 
        'プレイ前走者状況', 
        '捕手ID', 
        'opening_date', 'game_date',
        'start_time', 'game_time', 'elapsed_time'
    ], inplace=True)

### Rename

In [14]:
all_pitch.rename(columns={
    'データ内連番': 'No',
    '投球位置区域': 'pitch_area',
    '試合内連番': 'No_in_game',
    '試合内投球数': 'pitch_cnt_in_game',
    'イニング': 'inning',
    'イニング内打席数': 'bat_cnt_in_inning',
    '打席内投球数': 'pitch_cnt_in_bat',
    '投手登板順': 'pitch_order',
    '投手試合内対戦打者数': 'player_cnt_in_game',
    '投手試合内投球数': 'pitcher_cnt_in_game',
    '投手イニング内投球数': 'pitcher_cnt_in_inning',
    '打者打順': 'bat_order',
    '打者試合内打席数': 'bat_cnt_in_game',
    'プレイ前ホームチーム得点数': 'home_point',
    'プレイ前アウェイチーム得点数': 'away_point',
    'プレイ前アウト数': 'out_cnt',
    'プレイ前ボール数': 'ball_cnt',
    'プレイ前ストライク数': 'strike_cnt',
}, inplace=True)

In [15]:
print(all_pitch.shape)
all_pitch.head()

(778767, 78)


,No,ball,pitch_area,No_in_game,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,first,second,third,base_cnt,pitch_LR,bat_LR,top_bot,role,date_from_opening,elapsed_min,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,ball_cnt_2-0,ball_cnt_2-1,ball_cnt_2-2,ball_cnt_2-3,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,pit_bat_R_R,firm_pit,height_pit,weight_pit,draft_order_pit,salary_pit,foreigner_pit,company_pit,univ_pit,highsch_pit,age_pit,play_year_pit,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,foreigner_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0,0.000000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,196,100,NaN,24000,1,0,0,0,29,NaN,0,176,74,5.0,15000,0,1,1,0,32,8.0,0,178,83,1.0,5000,0,1,1,0,28,4.0
1,1,0.0,8.0,2,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0,0.200000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,196,100,NaN,24000,1,0,0,0,29,NaN,0,176,74,5.0,15000,0,1,1,0,32,8.0,0,178,83,1.0,5000,0,1,1,0,28,4.0
2,2,0.0,5.0,3,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0,0.383333,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,196,100,NaN,24000,1,0,0,0,29,NaN,0,176,74,5.0,15000,0,1,1,0,32,8.0,0,178,83,1.0,5000,0,1,1,0,28,4.0
3,3,0.0,12.0,4,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,1.016667,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,196,100,NaN,24000,1,0,0,0,29,NaN,0,180,74,1.0,7040,0,0,0,1,40,22.0,0,178,83,1.0,5000,0,1,1,0,28,4.0
4,4,0.0,8.0,5,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,1.200000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,196,100,NaN,24000,1,0,0,0,29,NaN,0,180,74,1.0,7040,0,0,0,1,40,22.0,0,178,83,1.0,5000,0,1,1,0,28,4.0


### train
- 行数: 257117

In [16]:
train = all_pitch.dropna(subset=['pitch_area'])
train.shape

(257117, 78)

### test
- 行数: 521650

In [17]:
test = all_pitch[all_pitch['pitch_area'].isnull()]
test.shape

(521650, 78)

In [18]:
del all_pitch
gc.collect()

64

In [19]:
train_d = train.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

In [20]:
test_d = test.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

## Dataset作成

In [21]:
lgb_train = lgb.Dataset(train_d, train['ball'])

### LGBM学習
- multiclass : クラス数=8

In [22]:
%%time
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 8,
        'num_leaves' : 31,
        'seed' : 0,
        'learning_rate' : 0.1,
    }

# 学習
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=8000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['multi_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
print('Best CV score:', cv_results['multi_logloss-mean'][-1])

[100]	cv_agg's multi_logloss: 1.20297 + 0.00221143
[200]	cv_agg's multi_logloss: 1.1717 + 0.00230705
[300]	cv_agg's multi_logloss: 1.15819 + 0.00235474
[400]	cv_agg's multi_logloss: 1.15096 + 0.00253513
[500]	cv_agg's multi_logloss: 1.1465 + 0.00239698
[600]	cv_agg's multi_logloss: 1.14396 + 0.00286288
[700]	cv_agg's multi_logloss: 1.14242 + 0.00336168
[800]	cv_agg's multi_logloss: 1.14184 + 0.00360873
[900]	cv_agg's multi_logloss: 1.14188 + 0.00371632
Best num_boost_round: 852
Best CV score: 1.1417108576925559
CPU times: user 1h 11s, sys: 46.9 s, total: 1h 58s
Wall time: 16min 6s


In [23]:
best_iter = int(num_boost_round * 1.1)

### 全体で再学習

In [24]:
%%time
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

CPU times: user 17min 30s, sys: 6.53 s, total: 17min 36s
Wall time: 4min 29s


### Feature Importance

In [25]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(10)

,feat_name,feat_imp
52,play_year_pit,7035
0,No_in_game,7628
57,salary_bat,8038
43,height_pit,8484
44,weight_pit,8897
25,elapsed_min,9012
8,pitcher_cnt_in_inning,9821
46,salary_pit,11510
7,pitcher_cnt_in_game,12484
24,date_from_opening,13433


In [26]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 12min 42s, sys: 1.96 s, total: 12min 44s
Wall time: 3min 16s


In [27]:
predict

array([[7.20401824e-01, 1.68751991e-02, 8.02152122e-02, ...,
        2.12809243e-04, 6.66096990e-05, 4.75830482e-03],
       [6.03313060e-01, 3.59916273e-02, 8.32164463e-02, ...,
        6.73280028e-04, 4.16781679e-06, 1.96292548e-03],
       [4.70887060e-01, 1.65423826e-01, 1.34120254e-01, ...,
        3.32976280e-03, 2.34970376e-04, 6.16420995e-03],
       ...,
       [4.37805855e-01, 6.40191944e-02, 3.14204474e-04, ...,
        1.44768066e-03, 1.25738656e-06, 6.97913015e-05],
       [5.43446623e-01, 6.60061813e-02, 3.29495707e-04, ...,
        9.80097338e-04, 2.14933363e-06, 6.22069431e-05],
       [5.17611057e-01, 5.45255381e-02, 4.54824474e-04, ...,
        2.46459314e-03, 1.64258442e-06, 9.43467173e-05]])

## 結果出力

In [28]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 9)


,index,0,1,2,3,4,5,6,7
0,0,0.720402,0.016875,0.080215,0.173992,0.003478,0.000213,0.000067,0.004758
1,1,0.603313,0.035992,0.083216,0.273051,0.001787,0.000673,0.000004,0.001963
2,2,0.470887,0.165424,0.134120,0.160976,0.058864,0.003330,0.000235,0.006164
3,3,0.460421,0.129014,0.161550,0.213539,0.029798,0.001691,0.000004,0.003982
4,4,0.351743,0.130184,0.305848,0.160598,0.042366,0.001428,0.000025,0.007806


### 2017の平均
- ストレート           0.468254
- カーブ             0.074725
- スライダー           0.185806
- シュート            0.070633
- フォーク            0.083013
- チェンジアップ         0.054217
- シンカー            0.011361
- カットボール          0.051992

In [29]:
submit.mean()

index    260824.500000
0             0.568200
1             0.068482
2             0.192697
3             0.043751
4             0.066215
5             0.038515
6             0.002098
7             0.020042
dtype: float64

In [31]:
submit.to_csv('submit/lgbm_3.csv', header=False, index=False)

### 評価結果
- CV score: 1.1417108576925559
- 評価結果  : 1.5951734